<a href="https://colab.research.google.com/github/shuntarotobari/qtest/blob/main/Growthtable.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd

# Load data
pwt = pd.read_stata('https://www.rug.nl/ggdc/docs/pwt100.dta')

# Filter and select relevant variables
countries = ['AUS', 'AUT', 'BEL', 'CAN', 'DNK', 'FIN', 'FRA', 'DEU', 'GRC', 'ISL', 'IRL', 'ITA', 'JPN', 'NLD', 'NZL', 'NOR', 'PRT', 'ESP', 'SWE', 'CHE', 'GBR', 'USA']
variables = ['countrycode', 'year', 'cgdpna', 'ckna', 'emp', 'avh', 'labsh']
data = pwt.loc[(pwt['countrycode'].isin(countries)) & (pwt['year'] >= 1960) & (pwt['year'] <= 2000), variables]

# Calculate additional variables
data = data.sort_values(['countrycode', 'year'])
data['alpha'] = 1 - data['labsh']
data['y_n'] = data['cgdpna'] / data['emp']
data['hours'] = data['emp'] * data['avh']
data['tfp_term'] = data.groupby('countrycode')['cgdpna'].pct_change().shift(-1) / (1 - data['alpha'])
data['cap_term'] = (data.groupby('countrycode')['ckna'] / data.groupby('countrycode')['cgdpna']).pct_change().shift(-1) * (data['alpha'] / (1 - data['alpha']))
data = data.dropna()

# Calculate growth rates
data = data.groupby('countrycode').apply(lambda x: x.assign(
    growth_rate=x['y_n'].pct_change(),
    tfp_growth=x['tfp_term'],
    capital_deepening=x['cap_term'],
    tfp_share=x['tfp_growth'] / x['growth_rate'],
    capital_share=1 - x['tfp_share']
)).reset_index(drop=True)

# Calculate averages
averages = data.groupby('year')[['growth_rate', 'tfp_growth', 'capital_deepening', 'tfp_share', 'capital_share']].mean().reset_index()
averages = averages.rename(columns={'countrycode': 'Country'})

# Print the table
print(pd.DataFrame({
    'Country': data['countrycode'].unique(),
    'Growth rate': data.groupby('countrycode')['growth_rate'].mean(),
    'TFP Growth': data.groupby('countrycode')['tfp_growth'].mean(),
    'Capital Deepening': data.groupby('countrycode')['capital_deepening'].mean(),
    'TFP share': data.groupby('countrycode')['tfp_share'].mean(),
    'Capital share': data.groupby('countrycode')['capital_share'].mean()
}).append(averages.set_index('Country'), sort=False).reset_index(drop=True))

KeyError: "['cgdpna', 'ckna'] not in index"